First and foremost, you should have already installed ParaView on your machine. If not, you can [download the package](https://www.paraview.org/download/) from [paraview.org](https://www.paraview.org).

ParaView is an open source platform that was specifically designed for data analysis and visualization. Briefly, ParaView is interesting (for us) because: 

  1. General purpose and can easily be used on laptop or HPC
  2. Naturally utilizes multiple processors for handling large data sets (in particular 3D)
  3. Includes a scripting interface (via Python)

Ultimately, we want to turn data (images, measurements, spectra, etc) into some rendering to represent the data in a way that provides insight or better understanding [for all viz, not just ParaView].

ParaView uses VTK, the Visualization Toolkit (more on this later).

VTK uses a basic data-flow paradigm, in which data flows through the system while being transformed at each step [via modules known as algorithms]. Algorithms have input ports to take data, and output ports to produce output:

  * Sources do not have input ports, but have outputs [i.e. to get data into the system, e.g., reading a file]
  * Sinks convert input into graphics such that they can be rendered 
  * Filters are intermediate algorithms, that convert input into output

Connecting sources, sinks, and filters can create arbitrarily complicated visual renderings. 

<img style="display: block; margin-left: auto; margin-right: auto" src="./Day2_images/pv_visualization_model.png" align="middle">

<div align="right"> <font size="-3">(credit: ParaView Guide) </font></div>

Following this brief introduction, we will start the visual exploration with ParaView. 

## Problem 1) Creating an Interactive Sphere

We will begin by creating a simple 3D object (a sphere). While there is nothing spectacular about a sphere, I'll note that it is not particularly easy to represent a(n interactive) sphere in `matplotlib`.

**Problem 1a**

Open paraview. 

Create a sphere. [*Sources $\rightarrow$ Sphere*]

At this stage you will notice that nothing is yet visible in the layout panel. However, a few things have happened.

There is now a pipeline module in the pipeline browser (upper left panel). Properties of the sphere can be adjusted in the properties panel (lower left). Finally, the *Apply* button in the properties panel is now active.

While this is not the case for this particular data set, because data ingestion and manipulation can be highly time consuming, ParaView allows you to perform those operations prior to rendering the data. 

The *Apply* allows you to accept those changes before proceeding. 

**Problem 1b**

Render the sphere [click *Apply*].

Use your mouse to inspect, rotate, and examine the sphere.

**Problem 1c**

Adjust the center of the sphere to the position [0,0,1], and adjust the radius to 1. 

**Problem 1d**

Make the sphere appear more "spherical" by adjusting theta resolution to 100. 

There are a variety of filters that can be applied to the data (in the *Filters* menu). The available filters are dependent on the data type provided to ParaView.

**Problem 1e**

Shrink the size of the mesh cells on the surface of the sphere. [*Filters* $\rightarrow$ *Alphabetical*  $\rightarrow$ *Shrink*]

Can you see the shrunken mesh cells?

## Problem 2) 


We will use the interactive version of ParaView to start (and we will use the python scripting tools later). 

As a starting place we are going to use 3D densitity and temperature data from ... simulations, which can be downloaded here: .

## Problem 1

**Problem 1a**

Using the File >> Open option, open the Density.vtk and Temperature.vtk files.

**Problem 1b**

Select the "eye" next to Density to "turn on" that data.

**Problem 1c**

Change the representation to Volume. Move around the cube to get a sense of the data.

**Problem 1d**

Select the eye for Temperature, and change the representation to Volume. Again, explore the cube.

**Problem 1e**

Change the colorbar for the Temperature. 

In the properties tab, under the Coloring header, select the choose presets folder (this is the folder with heart over it). 

Using your previous knowledge of visualization, select a terrible colorbar.

Replace the terrible colorbar with a useful colorbar.

**Problem 1f**

Change the colorbar for the Density.

**Problem 1g**

Turn on both the Temperature and Density. Can you see both volume renditions? If not, adjust the color schemes for each.


## Problem 2) Slicing the rendition

**Problem 2a**

Select the density data. Then slice the data using the tool in the top bar. When happy with the slice that you have selected, click "apply".

*Note* - you can adjust the position of the slice by dragging the outline, and you can adjust the angle of the slice with the arrow.

**Problem 2b**

Hide the density and temperature data, and examine the slice. If you are unhappy with your previous selection, adjust it.

**Problem 2c**

Add a contour to the slice. This can be done using the contour buttom (multiple concentric half-spheres).

Set the range for the contours from -2 to 2.5, and choose 6 steps for the contours. If you need to, adjust the color of the contours so they are visible relative to the 

**Problem 2d**

Add a clip to the temperature data. This can be done using the clip buttom (between the slice and contour buttons).

Adjust the position of the clip to match that of the density contour.

*Note* - you may need to slide the position of the clip slightly so that the contours remain visible.


